<a href="https://colab.research.google.com/github/galua2001/colab/blob/main/curve3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
using System.Collections.Generic;
using UnityEngine;

public class Draw : MonoBehaviour
{
    // 메인 스퀘어(첫 번째 생성된 스퀘어)를 지정하기 위한 변수
    public static Draw mainController = null;
    // 모든 인스턴스를 저장 (메인/복제본 구분)
    public static List<Draw> instances = new List<Draw>();
    // 메인 스퀘어에서 관리하는 전역 곡선 데이터 (정규화 좌표, 0~1)
    public static List<Vector2> globalCurve = new List<Vector2>();

    // 각 인스턴스가 사용할 곡선 데이터
    // 메인 스퀘어는 globalCurve를 참조하고, 복제본은 초기에는 globalCurve의 복사본을 사용
    private List<Vector2> localCurve = new List<Vector2>();

    public Color curveColor = Color.red;
    [Header("카메라 연결")]
    public Camera mainCamera;

    private LineRenderer line;

    void Awake()
    {
        if (!instances.Contains(this))
            instances.Add(this);

        // 아직 메인 컨트롤러가 지정되지 않았다면 첫 번째 생성된 객체가 메인 컨트롤러
        if (mainController == null)
            mainController = this;

        SetupLineRenderer();

        // 만약 Collider2D가 없다면 자동으로 추가 (Raycast를 위해 필요)
        if (GetComponent<Collider2D>() == null)
            gameObject.AddComponent<BoxCollider2D>();
    }

    void Start()
    {
        SpriteRenderer sr = GetComponent<SpriteRenderer>();
        if (sr == null)
        {
            Debug.LogError("SpriteRenderer가 필요합니다: " + name);
            return;
        }

        // 메인 스퀘어는 globalCurve를 사용하며, 없으면 기본 선을 추가
        if (this == mainController)
        {
            if (globalCurve.Count < 2)
            {
                // 예시 기본 선: 왼쪽 아래에서 오른쪽 위 (정규화 좌표 기준)
                globalCurve.Add(new Vector2(0.25f, 0.25f));
                globalCurve.Add(new Vector2(0.75f, 0.75f));
            }
            localCurve = globalCurve; // 참조 공유
        }
        else
        {
            // 복제본은 globalCurve의 복사본으로 시작
            localCurve = new List<Vector2>(globalCurve);
        }
        DrawCurve();
    }

    void OnDestroy()
    {
        instances.Remove(this);
        // 메인 스퀘어가 파괴되면 남은 인스턴스 중 첫 번째를 새 메인 컨트롤러로 지정
        if (this == mainController && instances.Count > 0)
            mainController = instances[0];
    }

    void Update()
    {
        if (mainCamera == null)
            return;

        Vector2 mouseWorld = mainCamera.ScreenToWorldPoint(Input.mousePosition);

        // 왼쪽 클릭 처리 (메인 스퀘어에서만 처리)
        if (Input.GetMouseButtonDown(0))
        {
            // Raycast로 클릭 대상이 이 스퀘어인지 체크
            RaycastHit2D hit = Physics2D.Raycast(mouseWorld, Vector2.zero);
            if (hit.collider != null && hit.collider.gameObject == this.gameObject)
            {
                // 왼쪽 클릭은 메인 스퀘어에서만 globalCurve에 점 추가
                if (this == mainController)
                {
                    Vector2 localPos = transform.InverseTransformPoint(mouseWorld);
                    Vector2 norm = ToNormalized(localPos);
                    if (IsInsideNormalized(norm))
                    {
                        globalCurve.Add(norm);
                        PropagateGlobal();
                    }
                    else
                    {
                        Debug.Log(name + ": 왼쪽 클릭 좌표가 정규화 범위 밖입니다: " + norm);
                    }
                }
                // 복제본은 왼쪽 클릭 무시
            }
        }

        // 오른쪽 클릭 처리
        if (Input.GetMouseButtonDown(1))
        {
            // Raycast로 클릭 대상이 이 스퀘어인지 체크
            RaycastHit2D hit = Physics2D.Raycast(mouseWorld, Vector2.zero);
            if (hit.collider != null && hit.collider.gameObject == this.gameObject)
            {
                Vector2 localPos = transform.InverseTransformPoint(mouseWorld);
                Vector2 normClick = ToNormalized(localPos);
                if (!IsInsideNormalized(normClick))
                {
                    Debug.Log(name + ": 우클릭 좌표가 정규화 범위 밖입니다: " + normClick);
                    return;
                }

                // 중앙(0.5, 0.5)을 기준으로 사분면 판정 (정규화 좌표)
                bool isLeft = normClick.x < 0.5f;
                bool isTop = normClick.y > 0.5f;

                if (this == mainController)
                {
                    // 메인 스퀘어의 우클릭은 globalCurve에 반영 후 모든 사각형에 전파
                    if (isLeft && !isTop)
                    {
                        // 왼쪽 아래: globalCurve 전체 삭제 (모든 사각형의 선 지움)
                        globalCurve.Clear();
                    }
                    else if (isLeft && isTop)
                    {
                        // 왼쪽 위: 180도 회전
                        Rotate180(globalCurve);
                    }
                    else if (!isLeft && isTop)
                    {
                        // 오른쪽 위: y축 대칭
                        MirrorY(globalCurve);
                    }
                    else // !isLeft && !isTop
                    {
                        // 오른쪽 아래: y=x 대칭
                        MirrorDiagonal(globalCurve);
                    }
                    PropagateGlobal();
                }
                else
                {
                    // 복제본의 우클릭은 자신의 localCurve만 수정 (다른 스퀘어에는 영향 없음)
                    if (isLeft && !isTop)
                    {
                        localCurve.Clear();
                    }
                    else if (isLeft && isTop)
                    {
                        Rotate180(localCurve);
                    }
                    else if (!isLeft && isTop)
                    {
                        MirrorY(localCurve);
                    }
                    else
                    {
                        MirrorDiagonal(localCurve);
                    }
                    DrawCurve();
                }
            }
        }
    }

    // localCurve를 이용하여 LineRenderer에 선 그리기
    void DrawCurve()
    {
        if (localCurve == null || localCurve.Count < 2)
        {
            line.positionCount = 0;
            return;
        }
        line.positionCount = localCurve.Count;
        for (int i = 0; i < localCurve.Count; i++)
        {
            Vector2 pos = FromNormalized(localCurve[i]);
            line.SetPosition(i, pos);
        }
    }

    // 메인 스퀘어의 globalCurve 변경 내용을 모든 인스턴스(복제본 포함)에 전파
    void PropagateGlobal()
    {
        foreach (var inst in instances)
        {
            if (inst == mainController)
            {
                inst.DrawCurve();
            }
            else
            {
                inst.localCurve = new List<Vector2>(globalCurve);
                inst.DrawCurve();
            }
        }
    }

    // 로컬 좌표 → 정규화 (0~1, 스프라이트 중심 기준)
    Vector2 ToNormalized(Vector2 localPos)
    {
        Vector2 size = GetSize();
        return new Vector2((localPos.x + size.x / 2f) / size.x, (localPos.y + size.y / 2f) / size.y);
    }

    // 정규화 좌표 → 로컬 좌표
    Vector2 FromNormalized(Vector2 norm)
    {
        Vector2 size = GetSize();
        return new Vector2(norm.x * size.x - size.x / 2f, norm.y * size.y - size.y / 2f);
    }

    // SpriteRenderer의 bounds.size 반환 (없으면 (1,1))
    Vector2 GetSize()
    {
        SpriteRenderer sr = GetComponent<SpriteRenderer>();
        if (sr != null)
            return sr.bounds.size;
        return Vector2.one;
    }

    // 정규화 좌표가 0~1 범위 내에 있는지 검사
    bool IsInsideNormalized(Vector2 norm)
    {
        return norm.x >= 0f && norm.x <= 1f && norm.y >= 0f && norm.y <= 1f;
    }

    // LineRenderer 초기화: 없으면 추가하고 기본 속성을 설정
    void SetupLineRenderer()
    {
        line = GetComponent<LineRenderer>();
        if (line == null)
            line = gameObject.AddComponent<LineRenderer>();

        line.startWidth = 0.05f;
        line.endWidth = 0.05f;
        line.material = new Material(Shader.Find("Sprites/Default"));
        line.sortingOrder = 100;
        line.startColor = curveColor;
        line.endColor = curveColor;
        line.useWorldSpace = false;
    }

    // 변환 함수들 (리스트에 대해 연산)

    // 180도 회전 (중심 기준)
    void Rotate180(List<Vector2> points)
    {
        for (int i = 0; i < points.Count; i++)
            points[i] = -points[i] + Vector2.one;
    }

    // y축 대칭 (x 좌표를 1 - x로 변경)
    void MirrorY(List<Vector2> points)
    {
        for (int i = 0; i < points.Count; i++)
        {
            Vector2 p = points[i];
            points[i] = new Vector2(1f - p.x, p.y);
        }
    }

    // y=x 대칭 (x와 y 값을 교환)
    void MirrorDiagonal(List<Vector2> points)
    {
        for (int i = 0; i < points.Count; i++)
        {
            Vector2 p = points[i];
            points[i] = new Vector2(p.y, p.x);
        }
    }
}
